Transformer

In [19]:


from scipy.special import softmax
import numpy as np

def test_gradient(dim, time_steps=50, scale=1.0):
    # Assume components of the query and keys are drawn from N(0, 1) independently
    q = np.random.randn(dim)
    ks = np.random.randn(time_steps, dim)
    x = np.sum(q * ks, axis=1) / scale  # x.shape = (time_steps,) 
    y = softmax(x)
    grad = np.diag(y) - np.outer(y, y)
    return np.max(np.abs(grad))  # the maximum component of gradients

NUMBER_OF_EXPERIMENTS = 5
# results of 5 random runs without scaling
print([test_gradient(100) for _ in range(NUMBER_OF_EXPERIMENTS)])
print([test_gradient(1000) for _ in range(NUMBER_OF_EXPERIMENTS)])

# results of 5 random runs with scaling
print([test_gradient(100, scale=np.sqrt(100)) for _ in range(NUMBER_OF_EXPERIMENTS)])
print([test_gradient(1000, scale=np.sqrt(1000)) for _ in range(NUMBER_OF_EXPERIMENTS)])

[0.02353202486135686, 0.03825262795920514, 0.001301216947797168, 0.24856292688104265, 0.15617721411082464]
[6.115357109592878e-10, 0.09666549769993338, 3.183231456205249e-12, 8.835204937662766e-07, 0.016844086122281388]
[0.07254786994253012, 0.08600248954152065, 0.05549865026325765, 0.07969999429142749, 0.22759543097204027]
[0.1631348895370225, 0.08097127589483341, 0.10276367098051566, 0.16719709509315875, 0.12023090695813965]


In [20]:
import torch
import torch.nn as nn
import os

class SelfAttention(nn.Module):
	def __init__(self, embed_size, heads):
		super(SelfAttention, self).__init__()
		self.embed_size = embed_size
		self.heads = heads
		self.head_dim = embed_size // heads

		assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"

		self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
		self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
		self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
		self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

	def forward(self, values, keys, query, mask):
		N =query.shape[0]
		value_len , key_len , query_len = values.shape[1], keys.shape[1], query.shape[1]

		# split embedding into self.heads pieces
		values = values.reshape(N, value_len, self.heads, self.head_dim)
		keys = keys.reshape(N, key_len, self.heads, self.head_dim)
		queries = query.reshape(N, query_len, self.heads, self.head_dim)
		
		values = self.values(values)
		keys = self.keys(keys)
		queries = self.queries(queries)

		energy = torch.einsum("nqhd,nkhd->nhqk", queries, keys)
		# queries shape: (N, query_len, heads, heads_dim)
		# keys shape : (N, key_len, heads, heads_dim)
		# energy shape: (N, heads, query_len, key_len)

		if mask is not None:
			energy = energy.masked_fill(mask == 0, float("-1e20"))

		attention = torch.softmax(energy/ (self.embed_size ** (1/2)), dim=3)

		out = torch.einsum("nhql, nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim)
		# attention shape: (N, heads, query_len, key_len)
		# values shape: (N, value_len, heads, heads_dim)
		# (N, query_len, heads, head_dim)

		out = self.fc_out(out)
		return out


class TransformerBlock(nn.Module):
	def __init__(self, embed_size, heads, dropout, forward_expansion):
		super(TransformerBlock, self).__init__()
		self.attention = SelfAttention(embed_size, heads)
		self.norm1 = nn.LayerNorm(embed_size)
		self.norm2 = nn.LayerNorm(embed_size)

		self.feed_forward = nn.Sequential(
			nn.Linear(embed_size, forward_expansion*embed_size),
			nn.ReLU(),
			nn.Linear(forward_expansion*embed_size, embed_size)
		)
		self.dropout = nn.Dropout(dropout)

	def forward(self, value, key, query, mask):
		attention = self.attention(value, key, query, mask)

		x = self.dropout(self.norm1(attention + query))
		forward = self.feed_forward(x)
		out = self.dropout(self.norm2(forward + x))
		return out


class Encoder(nn.Module):
	def __init__(
			self,
			src_vocab_size,
			embed_size,
			num_layers,
			heads,
			device,
			forward_expansion,
			dropout,
			max_length,
		):
		super(Encoder, self).__init__()
		self.embed_size = embed_size
		self.device = device
		self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
		self.position_embedding = nn.Embedding(max_length, embed_size)

		self.layers = nn.ModuleList(
			[
				TransformerBlock(
					embed_size,
					heads,
					dropout=dropout,
					forward_expansion=forward_expansion,
					)
				for _ in range(num_layers)]
		)
		self.dropout = nn.Dropout(dropout)


	def forward(self, x, mask):
		N, seq_length = x.shape
		positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
		out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
		for layer in self.layers:
			out = layer(out, out, out, mask)

		return out


class DecoderBlock(nn.Module):
	def __init__(self, embed_size, heads, forward_expansion, dropout, device):
		super(DecoderBlock, self).__init__()
		self.attention = SelfAttention(embed_size, heads)
		self.norm = nn.LayerNorm(embed_size)
		self.transformer_block = TransformerBlock(
			embed_size, heads, dropout, forward_expansion
		)

		self.dropout = nn.Dropout(dropout)

	def forward(self, x, value, key, src_mask, trg_mask):
		attention = self.attention(x, x, x, trg_mask)
		query = self.dropout(self.norm(attention + x))
		out = self.transformer_block(value, key, query, src_mask)
		return out

class Decoder(nn.Module):
	def __init__(
			self,
			trg_vocab_size,
			embed_size,
			num_layers,
			heads,
			forward_expansion,
			dropout,
			device,
			max_length,
	):
		super(Decoder, self).__init__()
		self.device = device
		self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
		self.position_embedding = nn.Embedding(max_length, embed_size)
		self.layers = nn.ModuleList(
			[DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
			for _ in range(num_layers)]
			)
		self.fc_out = nn.Linear(embed_size, trg_vocab_size)
		self.dropout = nn.Dropout(dropout)

	def forward(self, x ,enc_out , src_mask, trg_mask):
		N, seq_length = x.shape
		positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
		x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

		for layer in self.layers:
			x = layer(x, enc_out, enc_out, src_mask, trg_mask)

		out =self.fc_out(x)
		return out


class Transformer(nn.Module):
	def __init__(
			self,
			src_vocab_size,
			trg_vocab_size,
			src_pad_idx,
			trg_pad_idx,
			embed_size = 256,
			num_layers = 6,
			forward_expansion = 4,
			heads = 8,
			dropout = 0,
			device="cuda",
			max_length=100
		):
		super(Transformer, self).__init__()
		self.encoder = Encoder(
			src_vocab_size,
			embed_size,
			num_layers,
			heads,
			device,
			forward_expansion,
			dropout,
			max_length
			)
		self.decoder = Decoder(
			trg_vocab_size,
			embed_size,
			num_layers,
			heads,
			forward_expansion,
			dropout,
			device,
			max_length
			)


		self.src_pad_idx = src_pad_idx
		self.trg_pad_idx = trg_pad_idx
		self.device = device


	def make_src_mask(self, src):
		src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
		# (N, 1, 1, src_len)
		return src_mask.to(self.device)

	def make_trg_mask(self, trg):
		N, trg_len = trg.shape
		trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
			N, 1, trg_len, trg_len
		)
		return trg_mask.to(self.device)

	def forward(self, src, trg):
		src_mask = self.make_src_mask(src)
		trg_mask = self.make_trg_mask(trg)
		enc_src = self.encoder(src, src_mask)
		out = self.decoder(trg, enc_src, src_mask, trg_mask)
		return out


if __name__ == '__main__':
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(device)
	x = torch.tensor([[1,5,6,4,3,9,5,2,0],[1,8,7,3,4,5,6,7,2]]).to(device)
	trg = torch.tensor([[1,7,4,3,5,9,2,0],[1,5,6,2,4,7,6,2]]).to(device)

	src_pad_idx = 0
	trg_pad_idx = 0
	src_vocab_size = 10
	trg_vocab_size = 10
	model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)
	out = model(x, trg[:, : -1])
	print(out.shape)


cuda
torch.Size([2, 7, 10])


In [21]:
from torch import embedding


embedding=nn.Embedding(10,3)
#词表大小为10，词嵌入的维度为3
input1=torch.LongTensor([[1,2,4,5],[4,3,3,9]])

In [22]:
import math
from torch.autograd import Variable
from torch import nn
import torch


# 构建 Embedding 类来实现文本嵌入层
class Embedding(nn.Module):
    def __init__(self, vocab, d_model):
        # vocab： 词表的大小
        # d_model : 词嵌入的维度
        super(Embedding, self).__init__()
        # 定义 Embedding 层
        self.lut = nn.Embedding(vocab, d_model)
        # 将参数传入类中国
        self.d_model = d_model

    def forward(self, x):
        # x: 代表的是输入进模型的文本，通过词汇映射后的数字张量
        return self.lut(x) * math.sqrt(self.d_model) 


# 词表： 1000*512， 共是1000个词，每一行是一个词，每个词是一个512d的向量表示
vocab = 1000
d_model = 512

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
print("ember:", embr, "\n*********\n", embr.shape)


ember: tensor([[[-12.7855, -18.9745,  38.5586,  ...,  -0.3868, -42.8885, -30.1096],
         [ 13.6334, -21.2008, -16.9512,  ..., -17.3758, -15.3683,  13.4025],
         [  7.3512,  20.1956, -22.5922,  ...,  21.5400, -21.9771, -41.2900],
         [  2.6230,  17.0199,   2.0810,  ...,   5.1560, -10.6985,   7.5601]],

        [[ -8.1162,  56.3541,   0.6642,  ...,  33.7957,   1.4876,  -3.2103],
         [ 23.3764, -12.2481, -40.7027,  ...,  -6.1936, -49.8983,  13.4108],
         [  9.4422, -14.1022,   4.6149,  ...,  -0.3878,  -4.8056, -17.9027],
         [ -5.4352,   2.0048,   3.8061,  ...,   8.2661,   7.3232, -32.0618]]],
       grad_fn=<MulBackward0>) 
*********
 torch.Size([2, 4, 512])


In [23]:
import math
from torch.autograd import Variable
from torch import nn
import torch
from embedding_layer import Embedding

# 构建位置编码器的类
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        # d_model : 代表词嵌入的维度
        # dropout : 代表Dropout层的置零比率
        # max_len : 代表每个句子的最大长度
        super(PositionalEncoding, self).__init__()

        # 实例化 Dropout层
        self.dropout = nn.Dropout(p=dropout)

        # 初始化一个位置编码矩阵，大小是 max_len * d_model
        pe = torch.zeros(max_len, d_model)

        # 初始化一个绝对位置矩阵, max_len * 1
        position = torch.arange(0, max_len).unsqueeze(1)
        print(position)

        # 定义一个变化矩阵，div_term, 跳跃式的初始化
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        print("\ndiv_term", div_term)

        # 将前面定义的变化矩阵 进行技术，偶数分别赋值
        pe[:, 0::2] = torch.sin(position * div_term)  # 用正弦波给偶数部分赋值
        pe[:, 1::2] = torch.cos(position * div_term)  # 用余弦波给奇数部分赋值

        # 将二维张量，扩充为三维张量
        pe = pe.unsqueeze(0)  # 1 * max_len * d_model

        # 将位置编码矩阵，注册成模型的buffer，这个buffer不是模型中的参数，不跟随优化器同步更新
        # 注册成buffer后，就可以在模型保存后 重新加载的时候，将这个位置编码器和模型参数
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x : 代表文本序列的词嵌入表示
        # 首先明确pe的编码太长了，将第二个维度，就是max_len对应的维度，缩小成x的句子的同等的长度
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)  # 表示位置编码是不参与更新的
        return self.dropout(x)

d_model = 512
dropout = 0.1
max_len = 60
vocab = 1000

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
x = embr  # shape: [2, 4, 512]
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
print(pe_result)
print("\n*********\n", pe_result.shape)



ModuleNotFoundError: No module named 'embedding_layer'

In [ ]:
import matplotlib.pyplot as plt
from position import PositionalEncoding
import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np

plt.figure(figsize=(15, 5))

pe = PositionalEncoding(20, 0)  # 实例化这个模型，d_model = 20, dropout = 0
y = pe(Variable(torch.zeros(1, 100, 20)))  # 相当于只看位置矩阵

plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())

# 在画布上填写维度提示信息
plt.legend(["dim %d" %p for p in [4, 5, 6, 7]])
plt.show()


ModuleNotFoundError: No module named 'position'

In [ ]:
import torch
import numpy as np

def subsequent_mask(size):
    # size: 代表掩码张量 最后两个维度，形成一个方阵
    attn_shape = (1, size, size)

    # 使用np.ones()先构建一个全1 的张量，然后用np.triu形成上三角矩阵
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')

    # 反转
    return torch.from_numpy(1-subsequent_mask)

size = 5
sm = subsequent_mask(size)
print(sm)


tensor([[[1, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1]]], dtype=torch.uint8)


In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(subsequent_mask(20)[0])
plt.show()


: 

: 

In [ ]:
import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    # query, key, value : 代表注意力的三个输入张量
    # mask : 掩码张量
    # dropout : 传入Dropout实例化对象
    # 首先，将query的最后一个维度提取出来，代表的是词嵌入的维度
    d_k = query.size(-1)

    # 按照注意力计算公式，将query和key 的转置进行矩阵乘法，然后除以缩放系数
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # 判断是否使用掩码张量
    if mask is not None:
        # 利用masked_fill 方法，将掩码张量和0进行位置的意义比较，如果等于0，就替换成 -1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # scores的最后一个维度上进行 softmax
    p_attn = F.softmax(scores, dim=-1)

    # 判断是否使用dropout
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后一步完成p_attm 和 value 的乘法，并返回query的注意力表示
    return torch.matmul(p_attn, value), p_attn


query = key = value = pe_result
mask = Variable(torch.zeros(2, 4, 4))
attn, p_attn = attention(query, key, value, mask=mask)
print('attn', attn)
print('attn.shape', attn.shape)
print("p_attn", p_attn)
print(p_attn.shape)



NameError: name 'pe_result' is not defined

In [ ]:

def clones(module, N):
    # module : 代表要克隆的目标网络层
    # N : 将module几个
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


In [ ]:
import math
from torch.autograd import Variable
from torch import nn
import torch
from embedding_layer import Embedding
import copy

# 构建位置编码器的类
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        # d_model : 代表词嵌入的维度
        # dropout : 代表Dropout层的置零比率
        # max_len : 代表每个句子的最大长度
        super(PositionalEncoding, self).__init__()

        # 实例化 Dropout层
        self.dropout = nn.Dropout(p=dropout)

        # 初始化一个位置编码矩阵，大小是 max_len * d_model
        pe = torch.zeros(max_len, d_model)

        # 初始化一个绝对位置矩阵, max_len * 1
        position = torch.arange(0, max_len).unsqueeze(1)
        # print(position)

        # 定义一个变化矩阵，div_term, 跳跃式的初始化
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        # print("ndiv_term", div_term)

        # 将前面定义的变化矩阵 进行技术，偶数分别赋值
        pe[:, 0::2] = torch.sin(position * div_term)  # 用正弦波给偶数部分赋值
        pe[:, 1::2] = torch.cos(position * div_term)  # 用余弦波给奇数部分赋值

        # 将二维张量，扩充为三维张量
        pe = pe.unsqueeze(0)  # 1 * max_len * d_model

        # 将位置编码矩阵，注册成模型的buffer，这个buffer不是模型中的参数，不跟随优化器同步更新
        # 注册成buffer后，就可以在模型保存后 重新加载的时候，将这个位置编码器和模型参数
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x : 代表文本序列的词嵌入表示
        # 首先明确pe的编码太长了，将第二个维度，就是max_len对应的维度，缩小成x的句子的同等的长度
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)  # 表示位置编码是不参与更新的
        return self.dropout(x)


d_model = 512
dropout = 0.1
max_len = 60
vocab = 1000

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
x = embr  # shape: [2, 4, 512]
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
# print(pe_result)

import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    # query, key, value : 代表注意力的三个输入张量
    # mask : 掩码张量
    # dropout : 传入Dropout实例化对象
    # 首先，将query的最后一个维度提取出来，代表的是词嵌入的维度
    d_k = query.size(-1)

    # 按照注意力计算公式，将query和key 的转置进行矩阵乘法，然后除以缩放系数
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    print("..", scores.shape)
    # 判断是否使用掩码张量
    if mask is not None:
        # 利用masked_fill 方法，将掩码张量和0进行位置的意义比较，如果等于0，就替换成 -1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # scores的最后一个维度上进行 softmax
    p_attn = F.softmax(scores, dim=-1)

    # 判断是否使用dropout
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后一步完成p_attm 和 value 的乘法，并返回query的注意力表示
    return torch.matmul(p_attn, value), p_attn


query = key = value = pe_result
mask = Variable(torch.zeros(2, 4, 4))
attn, p_attn = attention(query, key, value, mask=mask)
# print('attn', attn)
# print('attn.shape', attn.shape)
# print("p_attn", p_attn)
# print(p_attn.shape)







# 实现克隆函数，因为在多头注意力机制下，要用到多个结果相同的线性层
# 需要使用clone 函数u，将他们统一 初始化到一个网络层列表对象中
def clones(module, N):
    # module : 代表要克隆的目标网络层
    # N : 将module几个
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


# 实现多头注意力机制的类
class MultiHeadAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout=0.1):
        # head : 代表几个头的函数
        # embedding_dim ： 代表词嵌入的维度
        # dropout
        super(MultiHeadAttention, self).__init__()

        # 强调：多头的数量head 需要整除 词嵌入的维度 embedding_dim
        assert embedding_dim % head == 0

        # 得到每个头，所获得 的词向量的维度
        self.d_k = embedding_dim // head
        self.head = head
        self.embedding_dim = embedding_dim

        # 获得线性层，需要获得4个，分别是Q K V 以及最终输出的线性层
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)

        # 初始化注意力张量
        self.attn = None

        # 初始化dropout对象
        self.drop = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        # query,key,value 是注意力机制的三个输入张量，mask代表掩码张量
        # 首先判断是否使用掩码张量
        if mask is not None:
            # 使用squeeze将掩码张量进行围堵扩充，代表多头的第n个头
            mask = mask.unsqueeze(1)

        # 得到batch_size
        batch_size = query.size(0)

        # 首先使用 zip 将网络能和输入数据连接在一起，模型的输出 利用 view 和 transpose 进行维度和形状的
        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1, 2)
             for model, x in zip(self.linears, (query, key, value))]

        # 将每个头的输出 传入到注意力层
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.drop)

        # 得到每个头的计算结果，每个output都是4维的张量，需要进行维度转换
        # 前面已经将transpose（1， 2）
        # 注意，先transpose 然后 contiguous，否则无法使用view
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head*self.d_k)

        # 最后将x输入到线性层的最后一个线性层中进行处理，得到最终的多头注意力结构输出
        return self.linears[-1](x)

# 实例化若干个参数
head = 8
embedding_dim = 512
dropout = 0.2

# 若干输入参数的初始化
query = key = value = pe_result

mask = Variable(torch.zeros(2, 4, 4))

mha = MultiHeadAttention(head, embedding_dim, dropout)
mha_result = mha(query, key, value, mask)

print(mha_result)
print(mha_result.shape)







ModuleNotFoundError: No module named 'embedding_layer'

In [ ]:
import math
from torch.autograd import Variable
from torch import nn
import torch
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


# main作用：集成了整个Transformer代码
########################################################################################################################

########################################################################################################################
# 构建 Embedding 类来实现文本嵌入层
# vocab : 词表的长度， d_model : 词嵌入的维度
class Embedding(nn.Module):
    def __init__(self, vocab, d_model):
        super(Embedding, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


# 词表： 1000*512， 共是1000个词，每一行是一个词，每个词是一个512d的向量表示
vocab = 1000
d_model = 512

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)

########################################################################################################################
# 构建位置编码器的类
# d_model : 代表词嵌入的维度
# dropout : 代表Dropout层的置零比率
# max_len : 代表每个句子的最大长度
# 初始化一个位置编码矩阵pe，大小是 max_len * d_model
# 初始化一个绝对位置矩阵position, 大小是max_len * 1
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        # 定义一个变化矩阵，div_term, 跳跃式的初始化
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        # 将前面定义的变化矩阵 进行技术，偶数分别赋值
        pe[:, 0::2] = torch.sin(position * div_term)  # 用正弦波给偶数部分赋值
        pe[:, 1::2] = torch.cos(position * div_term)  # 用余弦波给奇数部分赋值

        # 将二维张量，扩充为三维张量
        pe = pe.unsqueeze(0)  # 1 * max_len * d_model

        # 将位置编码矩阵，注册成模型的buffer，这个buffer不是模型中的参数，不跟随优化器同步更新
        # 注册成buffer后，就可以在模型保存后 重新加载的时候，将这个位置编码器和模型参数
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x : 代表文本序列的词嵌入表示
        # 首先明确pe的编码太长了，将第二个维度，就是max_len对应的维度，缩小成x的句子的同等的长度
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)  # 表示位置编码是不参与更新的
        return self.dropout(x)


d_model = 512
dropout = 0.1
max_len = 60
vocab = 1000

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
x = embr  # shape: [2, 4, 512]
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
# print(pe_result)



def attention(query, key, value, mask=None, dropout=None):
    # query, key, value : 代表注意力的三个输入张量
    # mask : 掩码张量
    # dropout : 传入Dropout实例化对象
    # 首先，将query的最后一个维度提取出来，代表的是词嵌入的维度
    d_k = query.size(-1)

    # 按照注意力计算公式，将query和key 的转置进行矩阵乘法，然后除以缩放系数
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # print("..", scores.shape)
    # 判断是否使用掩码张量
    if mask is not None:
        # 利用masked_fill 方法，将掩码张量和0进行位置的意义比较，如果等于0，就替换成 -1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # scores的最后一个维度上进行 softmax
    p_attn = F.softmax(scores, dim=-1)

    # 判断是否使用dropout
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后一步完成p_attm 和 value 的乘法，并返回query的注意力表示
    return torch.matmul(p_attn, value), p_attn


query = key = value = pe_result
mask = Variable(torch.zeros(2, 4, 4))
attn, p_attn = attention(query, key, value, mask=mask)
# print('attn', attn)
# print('attn.shape', attn.shape)
# print("p_attn", p_attn)
# print(p_attn.shape)


# 实现克隆函数，因为在多头注意力机制下，要用到多个结果相同的线性层
# 需要使用clone 函数u，将他们统一 初始化到一个网络层列表对象中
def clones(module, N):
    # module : 代表要克隆的目标网络层
    # N : 将module几个
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


# 实现多头注意力机制的类
class MultiHeadAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout=0.1):
        # head : 代表几个头的函数
        # embedding_dim ： 代表词嵌入的维度
        # dropout
        super(MultiHeadAttention, self).__init__()

        # 强调：多头的数量head 需要整除 词嵌入的维度 embedding_dim
        assert embedding_dim % head == 0

        # 得到每个头，所获得 的词向量的维度
        self.d_k = embedding_dim // head
        self.head = head
        self.embedding_dim = embedding_dim

        # 获得线性层，需要获得4个，分别是Q K V 以及最终输出的线性层
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)

        # 初始化注意力张量
        self.attn = None

        # 初始化dropout对象
        self.drop = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        # query,key,value 是注意力机制的三个输入张量，mask代表掩码张量
        # 首先判断是否使用掩码张量
        if mask is not None:
            # 使用squeeze将掩码张量进行围堵扩充，代表多头的第n个头
            mask = mask.unsqueeze(1)

        # 得到batch_size
        batch_size = query.size(0)

        # 首先使用 zip 将网络能和输入数据连接在一起，模型的输出 利用 view 和 transpose 进行维度和形状的
        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1, 2)
             for model, x in zip(self.linears, (query, key, value))]

        # 将每个头的输出 传入到注意力层
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.drop)

        # 得到每个头的计算结果，每个output都是4维的张量，需要进行维度转换
        # 前面已经将transpose（1， 2）
        # 注意，先transpose 然后 contiguous，否则无法使用view
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head*self.d_k)

        # 最后将x输入到线性层的最后一个线性层中进行处理，得到最终的多头注意力结构输出
        return self.linears[-1](x)

# 实例化若干个参数
head = 8
embedding_dim = 512
dropout = 0.2

# 若干输入参数的初始化
query = key = value = pe_result

mask = Variable(torch.zeros(2, 4, 4))

mha = MultiHeadAttention(head, embedding_dim, dropout)
mha_result = mha(query, key, value, mask)

# print(mha_result)
# print(mha_result.shape)


import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        # d_model : 代表词嵌入的维度，同时也是两个线性层的输入维度和输出维度
        # d_ff : 代表第一个线性层的输出维度，和第二个线性层的输入维度
        # dropout : 经过Dropout层处理时，随机置零
        super(PositionwiseFeedForward, self).__init__()

        # 定义两层全连接的线性层
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # x: 来自上一层的输出
        # 首先将x送入第一个线性网络，然后relu  然后dropout
        # 然后送入第二个线性层
        return self.w2(self.dropout(F.relu((self.w1(x)))))


d_model = 512
d_ff = 64
dropout = 0.2

# 这个是上一层的输出，作为前馈连接的输入
x = mha_result
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
ff_result = ff(x)
# print(ff_result)
# print(ff_result.shape)


# 构架规范化层的类
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        # features : 代表词嵌入的维度
        # eps ：一个很小的数，防止在规范化公式 除以0
        super(LayerNorm, self).__init__()
        # 初始化两个参数张量 a2 b 2 用于对结果作规范化 操作计算
        # 用nn.Parameter  封装，代表他们也是模型中的参数，也要随着模型计算而计算
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        self.eps = eps  # 传入到模型中去

    def forward(self, x):
        # x : 是上一层网络的输出 （两层的前馈全连接层）
        # 首先对x进行 最后一个维度上的求均值操作，同时要求保持输出维度和输入维度一致
        mean = x.mean(-1, keepdim=True)
        # 接着对x最后一个维度上求标准差的操作，同时要求保持输出维度和输入维度一制
        std = x.std(-1, keepdim=True)
        # 按照规范化公式进行计算并返回
        return self.a2 * (x-mean) / (std + self.eps) + self.b2


features = d_model = 512
eps = 1e-6

x = ff_result
ln = LayerNorm(features, eps)
ln_result = ln(x)
print(ln_result)
print(ln_result.shape)




tensor([[[ 0.5861, -0.0117,  1.2004,  ..., -0.6232,  0.3535,  1.3491],
         [ 0.8099, -0.5379,  0.8348,  ..., -1.5828, -0.6371,  0.9689],
         [-1.1210, -0.2339,  1.6161,  ..., -1.4314,  0.8251,  1.0368],
         [-0.1154, -0.0739,  1.6003,  ..., -0.8956,  0.2955,  1.7212]],

        [[ 0.1560, -0.4482,  1.1628,  ..., -0.6699, -0.3188,  1.3650],
         [ 0.2348, -0.1948,  1.3426,  ..., -1.5506,  0.2838,  1.4310],
         [ 1.1146,  0.2120, -0.5381,  ..., -0.9989,  0.0524,  0.5629],
         [ 0.6154, -0.2227,  0.8482,  ...,  0.3371, -0.7661,  0.6899]]],
       grad_fn=<AddBackward0>)
torch.Size([2, 4, 512])


In [ ]:
import math
from torch.autograd import Variable
from torch import nn
import torch
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


# main作用：集成了整个Transformer代码
########################################################################################################################

########################################################################################################################
# 构建 Embedding 类来实现文本嵌入层
# vocab : 词表的长度， d_model : 词嵌入的维度
class Embedding(nn.Module):
    def __init__(self, vocab, d_model):
        super(Embedding, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


# 词表： 1000*512， 共是1000个词，每一行是一个词，每个词是一个512d的向量表示
vocab = 1000
d_model = 512

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)

########################################################################################################################
# 构建位置编码器的类
# d_model : 代表词嵌入的维度
# dropout : 代表Dropout层的置零比率
# max_len : 代表每个句子的最大长度
# 初始化一个位置编码矩阵pe，大小是 max_len * d_model
# 初始化一个绝对位置矩阵position, 大小是max_len * 1
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        # 定义一个变化矩阵，div_term, 跳跃式的初始化
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        # 将前面定义的变化矩阵 进行技术，偶数分别赋值
        pe[:, 0::2] = torch.sin(position * div_term)  # 用正弦波给偶数部分赋值
        pe[:, 1::2] = torch.cos(position * div_term)  # 用余弦波给奇数部分赋值

        # 将二维张量，扩充为三维张量
        pe = pe.unsqueeze(0)  # 1 * max_len * d_model

        # 将位置编码矩阵，注册成模型的buffer，这个buffer不是模型中的参数，不跟随优化器同步更新
        # 注册成buffer后，就可以在模型保存后 重新加载的时候，将这个位置编码器和模型参数
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x : 代表文本序列的词嵌入表示
        # 首先明确pe的编码太长了，将第二个维度，就是max_len对应的维度，缩小成x的句子的同等的长度
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)  # 表示位置编码是不参与更新的
        return self.dropout(x)


d_model = 512
dropout = 0.1
max_len = 60
vocab = 1000

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
x = embr  # shape: [2, 4, 512]
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
# print(pe_result)



def attention(query, key, value, mask=None, dropout=None):
    # query, key, value : 代表注意力的三个输入张量
    # mask : 掩码张量
    # dropout : 传入Dropout实例化对象
    # 首先，将query的最后一个维度提取出来，代表的是词嵌入的维度
    d_k = query.size(-1)

    # 按照注意力计算公式，将query和key 的转置进行矩阵乘法，然后除以缩放系数
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # print("..", scores.shape)
    # 判断是否使用掩码张量
    if mask is not None:
        # 利用masked_fill 方法，将掩码张量和0进行位置的意义比较，如果等于0，就替换成 -1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # scores的最后一个维度上进行 softmax
    p_attn = F.softmax(scores, dim=-1)

    # 判断是否使用dropout
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后一步完成p_attm 和 value 的乘法，并返回query的注意力表示
    return torch.matmul(p_attn, value), p_attn


query = key = value = pe_result
mask = Variable(torch.zeros(2, 4, 4))
attn, p_attn = attention(query, key, value, mask=mask)
# print('attn', attn)
# print('attn.shape', attn.shape)
# print("p_attn", p_attn)
# print(p_attn.shape)


# 实现克隆函数，因为在多头注意力机制下，要用到多个结果相同的线性层
# 需要使用clone 函数u，将他们统一 初始化到一个网络层列表对象中
def clones(module, N):
    # module : 代表要克隆的目标网络层
    # N : 将module几个
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


# 实现多头注意力机制的类
class MultiHeadAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout=0.1):
        # head : 代表几个头的函数
        # embedding_dim ： 代表词嵌入的维度
        # dropout
        super(MultiHeadAttention, self).__init__()

        # 强调：多头的数量head 需要整除 词嵌入的维度 embedding_dim
        assert embedding_dim % head == 0

        # 得到每个头，所获得 的词向量的维度
        self.d_k = embedding_dim // head
        self.head = head
        self.embedding_dim = embedding_dim

        # 获得线性层，需要获得4个，分别是Q K V 以及最终输出的线性层
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)

        # 初始化注意力张量
        self.attn = None

        # 初始化dropout对象
        self.drop = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        # query,key,value 是注意力机制的三个输入张量，mask代表掩码张量
        # 首先判断是否使用掩码张量
        if mask is not None:
            # 使用squeeze将掩码张量进行围堵扩充，代表多头的第n个头
            mask = mask.unsqueeze(1)

        # 得到batch_size
        batch_size = query.size(0)

        # 首先使用 zip 将网络能和输入数据连接在一起，模型的输出 利用 view 和 transpose 进行维度和形状的
        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1, 2)
             for model, x in zip(self.linears, (query, key, value))]

        # 将每个头的输出 传入到注意力层
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.drop)

        # 得到每个头的计算结果，每个output都是4维的张量，需要进行维度转换
        # 前面已经将transpose（1， 2）
        # 注意，先transpose 然后 contiguous，否则无法使用view
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head*self.d_k)

        # 最后将x输入到线性层的最后一个线性层中进行处理，得到最终的多头注意力结构输出
        return self.linears[-1](x)

# 实例化若干个参数
head = 8
embedding_dim = 512
dropout = 0.2

# 若干输入参数的初始化
query = key = value = pe_result

mask = Variable(torch.zeros(2, 4, 4))

mha = MultiHeadAttention(head, embedding_dim, dropout)
mha_result = mha(query, key, value, mask)

# print(mha_result)
# print(mha_result.shape)


import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        # d_model : 代表词嵌入的维度，同时也是两个线性层的输入维度和输出维度
        # d_ff : 代表第一个线性层的输出维度，和第二个线性层的输入维度
        # dropout : 经过Dropout层处理时，随机置零
        super(PositionwiseFeedForward, self).__init__()

        # 定义两层全连接的线性层
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # x: 来自上一层的输出
        # 首先将x送入第一个线性网络，然后relu  然后dropout
        # 然后送入第二个线性层
        return self.w2(self.dropout(F.relu((self.w1(x)))))


d_model = 512
d_ff = 64
dropout = 0.2

# 这个是上一层的输出，作为前馈连接的输入
x = mha_result
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
ff_result = ff(x)
# print(ff_result)
# print(ff_result.shape)


# 构架规范化层的类
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        # features : 代表词嵌入的维度
        # eps ：一个很小的数，防止在规范化公式 除以0
        super(LayerNorm, self).__init__()
        # 初始化两个参数张量 a2 b 2 用于对结果作规范化 操作计算
        # 用nn.Parameter  封装，代表他们也是模型中的参数，也要随着模型计算而计算
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        self.eps = eps  # 传入到模型中去

    def forward(self, x):
        # x : 是上一层网络的输出 （两层的前馈全连接层）
        # 首先对x进行 最后一个维度上的求均值操作，同时要求保持输出维度和输入维度一致
        mean = x.mean(-1, keepdim=True)
        # 接着对x最后一个维度上求标准差的操作，同时要求保持输出维度和输入维度一制
        std = x.std(-1, keepdim=True)
        # 按照规范化公式进行计算并返回
        return self.a2 * (x-mean) / (std + self.eps) + self.b2


features = d_model = 512
eps = 1e-6

x = ff_result
ln = LayerNorm(features, eps)
ln_result = ln(x)
# print(ln_result)
# print(ln_result.shape)

# 构建子层连接结构的类
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout=0.1):
        # size 是词嵌入的维度
        super(SublayerConnection, self).__init__()
        # 实例化一个规范化层的对象
        self.norm = LayerNorm(size)\
        # 实例化一个dropout对象
        self.dropout = nn.Dropout(p=dropout)
        self.size = size

    def forward(self, x, sublayer):
        # : x代表上一层传入的张量
        # sublayer ： 代表子层连接中 子层函数
        # 首先将x进行规范化，送入子层函数，然后dropout， 最后残差连接
        return x + self.dropout(sublayer(self.norm(x)))

size = d_model = 512
head = 8
dropout = 0.2

x = pe_result
mask = Variable(torch.zeros(2, 4, 4))
# 子层函数采用的是多头注意力机制
self_attn = MultiHeadAttention(head, d_model)

sublayer = lambda x: self_attn(x, x, x, mask)

sc = SublayerConnection(size, dropout)
sc_result = sc(x, sublayer)
# print(sc_result)
# print(sc_result.shape)


# 构建编码器层的类
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        # size : 代表词嵌入的维度
        # self_attn : 代表传入的多头自注意力子层的实例化对象
        # feed_forward : 代表前馈全连接层实例化对象
        # dropout : 进行dropout置零比率
        super(EncoderLayer, self).__init__()

        # 将两个实例化对象和参数传入类中
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = size

        # 编码器层中，有两个子层连接结构，需要clones函数进行操作
        self.sublayer = clones(SublayerConnection(size, dropout), 2)

    def forward(self, x, mask):
        # x: 代表上一层传入的张量（位置编码
        # mask  ： 代表掩码张量
        # 首先让 x 经过第一个子层连接结构，内部包含多头自注意力机制子层
        # 再让张量经过第二个子层连接结构，其中包含前馈全连接网络
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


size = d_model = 512
head = 8
d_ff = 64
x = pe_result
dropout = 0.2
self_attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
mask = Variable(torch.zeros(2, 4, 4))

el = EncoderLayer(size, self_attn, ff, dropout)
el_result = el(x, mask)
# print(el_result)
# print(el_result.shape)


# 构建编码器类 Encoder
class Encoder(nn.Module):
    def __init__(self, layer, N):
        # layer : 代表上一节编写的 编码器层
        # N ： 代表 编码器中需要 几个编码器层（layer）
        super(Encoder, self).__init__()
        # 首先使用 clones  函数 克隆 N 个编码器层  防止在self.layer中
        self.layers = clones(layer, N)
        # 初始化一个规范化层，作用在编码器后面
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        # 代表上一层输出的张量
        # mask 是掩码张量
        # 让x 依次经过N个编码器层的处理；最后再经过规范化层就可以输出了
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


size = d_model = 512
head = 8
d_ff = 64
c = copy.deepcopy
attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
dropout = 0.2
layer = EncoderLayer(size, c(attn), c(ff), dropout)
N = 8
mask = Variable(torch.zeros(2, 4, 4))
en = Encoder(layer, N)
en_result = en(x, mask)
# print(en_result)
# print(en_result.shape)


# 构建解码器层类
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        # size ： 代表词嵌入的维度
        # attn ：多头自注意力机制对象
        # src_attn : 常规的注意力机制对象
        # feed_forweawrd : 前馈全连接层
        super(DecoderLayer, self).__init__()

        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.dropout = dropout

        # 使用clones函数， 克隆3个子层连接对象
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, source_mask, target_mask):
        # x: 上一层输入的张量
        # memory ： 代表编码器的语义存储张量
        # source_mask : 源数据的掩码张量
        # target_mask : 目标数据的掩码张量
        m = memory

        # 第一步，让x 进入第一个子层，（多头自注意力子层
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, target_mask))

        # 第二步，让x 进入第二个子层，（常规注意力子层
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, source_mask))

        # 第三步，让x进入第三个子层，前馈全连接层
        return self.sublayer[2](x, self.feed_forward)


size = d_model = 512
head = 8
d_ff = 64
dropout = 0.2

# 这里就没有区分多头自注意力和常规注意力机制了
self_attn = src_attn = MultiHeadAttention(head, d_model, dropout)
ff  = PositionwiseFeedForward(d_model, d_ff, dropout)
x = pe_result
memory = en_result
mask = Variable(torch.zeros(2, 4, 4))
source_mask = target_mask = mask

dl = DecoderLayer(size, self_attn, src_attn, ff, dropout)
dl_result = dl(x, memory, source_mask, target_mask)
# print(dl_result)
# print(dl_result.shape)


# 构建解码器类
class Decoder(nn.Module):
    def __init__(self, layer, N):
        # layer : 代表解码器层 的对象
        # N : 代表将layer进行几层的拷贝
        super(Decoder, self).__init__()
        # 利用clones函数克隆N个layer
        self.layers = clones(layer, N)
        # 实例化一个规范化层
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, source_mask, target_mask):
        # x: 代表目标数数的嵌入表示
        # memory : 代表编码器的输出张量
        # source_mask ： 源数据的掩码张量
        # target_mask: 目标数据的掩码张量
        # x经过所有的编码器层，最后通过规范化层
        for layer in self.layers:
            x = layer(x, memory, source_mask, target_mask)
        return self.norm(x)


size = d_model = 512
head = 8
d_ff = 64
dropout = 0.2
c = copy.deepcopy
attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
layer = DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout)

N = 8
x = pe_result
memory = en_result
mask = Variable(torch.zeros(2, 4, 4))
source_mask = target_mask = mask

de = Decoder(layer, N)
de_result = de(x, memory, source_mask, target_mask)
print(de_result)
print(de_result.shape)





tensor([[[-0.4029,  2.0160, -2.5725,  ..., -0.0695,  0.3837, -0.7650],
         [-1.8618,  0.1176,  1.5201,  ..., -0.6094,  1.7964, -0.4402],
         [-1.4517, -0.7813,  0.3225,  ..., -0.9868, -0.0330,  0.4345],
         [-0.2051, -1.0454, -0.0742,  ...,  1.5572, -0.1397, -0.9129]],

        [[-0.7303,  0.1544,  1.1549,  ..., -1.3101,  0.7498, -0.9943],
         [-1.1328,  0.8129,  0.3164,  ...,  1.0723, -0.2347,  0.9507],
         [-0.0599, -0.1497,  0.7505,  ..., -1.8553,  1.7566, -0.0384],
         [-0.5869,  0.1655, -0.9241,  ..., -1.0236,  0.4136, -0.9395]]],
       grad_fn=<AddBackward0>)
torch.Size([2, 4, 512])


In [ ]:
import math
from torch.autograd import Variable
from torch import nn
import torch
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


# main作用：集成了整个Transformer代码
########################################################################################################################

########################################################################################################################
# 构建 Embedding 类来实现文本嵌入层
# vocab : 词表的长度， d_model : 词嵌入的维度
class Embedding(nn.Module):
    def __init__(self, vocab, d_model):
        super(Embedding, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)


# 词表： 1000*512， 共是1000个词，每一行是一个词，每个词是一个512d的向量表示
vocab = 1000
d_model = 512

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)

########################################################################################################################
# 构建位置编码器的类
# d_model : 代表词嵌入的维度
# dropout : 代表Dropout层的置零比率
# max_len : 代表每个句子的最大长度
# 初始化一个位置编码矩阵pe，大小是 max_len * d_model
# 初始化一个绝对位置矩阵position, 大小是max_len * 1
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        # 定义一个变化矩阵，div_term, 跳跃式的初始化
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        # 将前面定义的变化矩阵 进行技术，偶数分别赋值
        pe[:, 0::2] = torch.sin(position * div_term)  # 用正弦波给偶数部分赋值
        pe[:, 1::2] = torch.cos(position * div_term)  # 用余弦波给奇数部分赋值

        # 将二维张量，扩充为三维张量
        pe = pe.unsqueeze(0)  # 1 * max_len * d_model

        # 将位置编码矩阵，注册成模型的buffer，这个buffer不是模型中的参数，不跟随优化器同步更新
        # 注册成buffer后，就可以在模型保存后 重新加载的时候，将这个位置编码器和模型参数
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x : 代表文本序列的词嵌入表示
        # 首先明确pe的编码太长了，将第二个维度，就是max_len对应的维度，缩小成x的句子的同等的长度
        x = x + Variable(self.pe[:, : x.size(1)], requires_grad=False)  # 表示位置编码是不参与更新的
        return self.dropout(x)


d_model = 512
dropout = 0.1
max_len = 60
vocab = 1000

x = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))

emb = Embedding(vocab, d_model)
embr = emb(x)
x = embr  # shape: [2, 4, 512]
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
# print(pe_result)



def attention(query, key, value, mask=None, dropout=None):
    # query, key, value : 代表注意力的三个输入张量
    # mask : 掩码张量
    # dropout : 传入Dropout实例化对象
    # 首先，将query的最后一个维度提取出来，代表的是词嵌入的维度
    d_k = query.size(-1)

    # 按照注意力计算公式，将query和key 的转置进行矩阵乘法，然后除以缩放系数
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    # print("..", scores.shape)
    # 判断是否使用掩码张量
    if mask is not None:
        # 利用masked_fill 方法，将掩码张量和0进行位置的意义比较，如果等于0，就替换成 -1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # scores的最后一个维度上进行 softmax
    p_attn = F.softmax(scores, dim=-1)

    # 判断是否使用dropout
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后一步完成p_attm 和 value 的乘法，并返回query的注意力表示
    return torch.matmul(p_attn, value), p_attn


query = key = value = pe_result
mask = Variable(torch.zeros(2, 4, 4))
attn, p_attn = attention(query, key, value, mask=mask)
# print('attn', attn)
# print('attn.shape', attn.shape)
# print("p_attn", p_attn)
# print(p_attn.shape)


# 实现克隆函数，因为在多头注意力机制下，要用到多个结果相同的线性层
# 需要使用clone 函数u，将他们统一 初始化到一个网络层列表对象中
def clones(module, N):
    # module : 代表要克隆的目标网络层
    # N : 将module几个
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


# 实现多头注意力机制的类
class MultiHeadAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout=0.1):
        # head : 代表几个头的函数
        # embedding_dim ： 代表词嵌入的维度
        # dropout
        super(MultiHeadAttention, self).__init__()

        # 强调：多头的数量head 需要整除 词嵌入的维度 embedding_dim
        assert embedding_dim % head == 0

        # 得到每个头，所获得 的词向量的维度
        self.d_k = embedding_dim // head
        self.head = head
        self.embedding_dim = embedding_dim

        # 获得线性层，需要获得4个，分别是Q K V 以及最终输出的线性层
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)

        # 初始化注意力张量
        self.attn = None

        # 初始化dropout对象
        self.drop = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        # query,key,value 是注意力机制的三个输入张量，mask代表掩码张量
        # 首先判断是否使用掩码张量
        if mask is not None:
            # 使用squeeze将掩码张量进行围堵扩充，代表多头的第n个头
            mask = mask.unsqueeze(1)

        # 得到batch_size
        batch_size = query.size(0)

        # 首先使用 zip 将网络能和输入数据连接在一起，模型的输出 利用 view 和 transpose 进行维度和形状的
        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1, 2)
             for model, x in zip(self.linears, (query, key, value))]

        # 将每个头的输出 传入到注意力层
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.drop)

        # 得到每个头的计算结果，每个output都是4维的张量，需要进行维度转换
        # 前面已经将transpose（1， 2）
        # 注意，先transpose 然后 contiguous，否则无法使用view
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head*self.d_k)

        # 最后将x输入到线性层的最后一个线性层中进行处理，得到最终的多头注意力结构输出
        return self.linears[-1](x)

# 实例化若干个参数
head = 8
embedding_dim = 512
dropout = 0.2

# 若干输入参数的初始化
query = key = value = pe_result

mask = Variable(torch.zeros(2, 4, 4))

mha = MultiHeadAttention(head, embedding_dim, dropout)
mha_result = mha(query, key, value, mask)

# print(mha_result)
# print(mha_result.shape)


import math
from torch.autograd import Variable
from torch import nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        # d_model : 代表词嵌入的维度，同时也是两个线性层的输入维度和输出维度
        # d_ff : 代表第一个线性层的输出维度，和第二个线性层的输入维度
        # dropout : 经过Dropout层处理时，随机置零
        super(PositionwiseFeedForward, self).__init__()

        # 定义两层全连接的线性层
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # x: 来自上一层的输出
        # 首先将x送入第一个线性网络，然后relu  然后dropout
        # 然后送入第二个线性层
        return self.w2(self.dropout(F.relu((self.w1(x)))))


d_model = 512
d_ff = 64
dropout = 0.2

# 这个是上一层的输出，作为前馈连接的输入
x = mha_result
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
ff_result = ff(x)
# print(ff_result)
# print(ff_result.shape)


# 构架规范化层的类
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        # features : 代表词嵌入的维度
        # eps ：一个很小的数，防止在规范化公式 除以0
        super(LayerNorm, self).__init__()
        # 初始化两个参数张量 a2 b 2 用于对结果作规范化 操作计算
        # 用nn.Parameter  封装，代表他们也是模型中的参数，也要随着模型计算而计算
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        self.eps = eps  # 传入到模型中去

    def forward(self, x):
        # x : 是上一层网络的输出 （两层的前馈全连接层）
        # 首先对x进行 最后一个维度上的求均值操作，同时要求保持输出维度和输入维度一致
        mean = x.mean(-1, keepdim=True)
        # 接着对x最后一个维度上求标准差的操作，同时要求保持输出维度和输入维度一制
        std = x.std(-1, keepdim=True)
        # 按照规范化公式进行计算并返回
        return self.a2 * (x-mean) / (std + self.eps) + self.b2


features = d_model = 512
eps = 1e-6

x = ff_result
ln = LayerNorm(features, eps)
ln_result = ln(x)
# print(ln_result)
# print(ln_result.shape)

# 构建子层连接结构的类
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout=0.1):
        # size 是词嵌入的维度
        super(SublayerConnection, self).__init__()
        # 实例化一个规范化层的对象
        self.norm = LayerNorm(size)\
        # 实例化一个dropout对象
        self.dropout = nn.Dropout(p=dropout)
        self.size = size

    def forward(self, x, sublayer):
        # : x代表上一层传入的张量
        # sublayer ： 代表子层连接中 子层函数
        # 首先将x进行规范化，送入子层函数，然后dropout， 最后残差连接
        return x + self.dropout(sublayer(self.norm(x)))

size = d_model = 512
head = 8
dropout = 0.2

x = pe_result
mask = Variable(torch.zeros(2, 4, 4))
# 子层函数采用的是多头注意力机制
self_attn = MultiHeadAttention(head, d_model)

sublayer = lambda x: self_attn(x, x, x, mask)

sc = SublayerConnection(size, dropout)
sc_result = sc(x, sublayer)
# print(sc_result)
# print(sc_result.shape)


# 构建编码器层的类
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        # size : 代表词嵌入的维度
        # self_attn : 代表传入的多头自注意力子层的实例化对象
        # feed_forward : 代表前馈全连接层实例化对象
        # dropout : 进行dropout置零比率
        super(EncoderLayer, self).__init__()

        # 将两个实例化对象和参数传入类中
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = size

        # 编码器层中，有两个子层连接结构，需要clones函数进行操作
        self.sublayer = clones(SublayerConnection(size, dropout), 2)

    def forward(self, x, mask):
        # x: 代表上一层传入的张量（位置编码
        # mask  ： 代表掩码张量
        # 首先让 x 经过第一个子层连接结构，内部包含多头自注意力机制子层
        # 再让张量经过第二个子层连接结构，其中包含前馈全连接网络
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


size = d_model = 512
head = 8
d_ff = 64
x = pe_result
dropout = 0.2
self_attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
mask = Variable(torch.zeros(2, 4, 4))

el = EncoderLayer(size, self_attn, ff, dropout)
el_result = el(x, mask)
# print(el_result)
# print(el_result.shape)


# 构建编码器类 Encoder
class Encoder(nn.Module):
    def __init__(self, layer, N):
        # layer : 代表上一节编写的 编码器层
        # N ： 代表 编码器中需要 几个编码器层（layer）
        super(Encoder, self).__init__()
        # 首先使用 clones  函数 克隆 N 个编码器层  防止在self.layer中
        self.layers = clones(layer, N)
        # 初始化一个规范化层，作用在编码器后面
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        # 代表上一层输出的张量
        # mask 是掩码张量
        # 让x 依次经过N个编码器层的处理；最后再经过规范化层就可以输出了
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


size = d_model = 512
head = 8
d_ff = 64
c = copy.deepcopy
attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
dropout = 0.2
layer = EncoderLayer(size, c(attn), c(ff), dropout)
N = 8
mask = Variable(torch.zeros(2, 4, 4))
en = Encoder(layer, N)
en_result = en(x, mask)
# print(en_result)
# print(en_result.shape)


# 构建解码器层类
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        # size ： 代表词嵌入的维度
        # attn ：多头自注意力机制对象
        # src_attn : 常规的注意力机制对象
        # feed_forweawrd : 前馈全连接层
        super(DecoderLayer, self).__init__()

        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.dropout = dropout

        # 使用clones函数， 克隆3个子层连接对象
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, source_mask, target_mask):
        # x: 上一层输入的张量
        # memory ： 代表编码器的语义存储张量
        # source_mask : 源数据的掩码张量
        # target_mask : 目标数据的掩码张量
        m = memory

        # 第一步，让x 进入第一个子层，（多头自注意力子层
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, target_mask))

        # 第二步，让x 进入第二个子层，（常规注意力子层
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, source_mask))

        # 第三步，让x进入第三个子层，前馈全连接层
        return self.sublayer[2](x, self.feed_forward)


size = d_model = 512
head = 8
d_ff = 64
dropout = 0.2

# 这里就没有区分多头自注意力和常规注意力机制了
self_attn = src_attn = MultiHeadAttention(head, d_model, dropout)
ff  = PositionwiseFeedForward(d_model, d_ff, dropout)
x = pe_result
memory = en_result
mask = Variable(torch.zeros(2, 4, 4))
source_mask = target_mask = mask

dl = DecoderLayer(size, self_attn, src_attn, ff, dropout)
dl_result = dl(x, memory, source_mask, target_mask)
# print(dl_result)
# print(dl_result.shape)


# 构建解码器类
class Decoder(nn.Module):
    def __init__(self, layer, N):
        # layer : 代表解码器层 的对象
        # N : 代表将layer进行几层的拷贝
        super(Decoder, self).__init__()
        # 利用clones函数克隆N个layer
        self.layers = clones(layer, N)
        # 实例化一个规范化层
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, source_mask, target_mask):
        # x: 代表目标数数的嵌入表示
        # memory : 代表编码器的输出张量
        # source_mask ： 源数据的掩码张量
        # target_mask: 目标数据的掩码张量
        # x经过所有的编码器层，最后通过规范化层
        for layer in self.layers:
            x = layer(x, memory, source_mask, target_mask)
        return self.norm(x)


size = d_model = 512
head = 8
d_ff = 64
dropout = 0.2
c = copy.deepcopy
attn = MultiHeadAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
layer = DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout)

N = 8
x = pe_result
memory = en_result
mask = Variable(torch.zeros(2, 4, 4))
source_mask = target_mask = mask

de = Decoder(layer, N)
de_result = de(x, memory, source_mask, target_mask)
# print(de_result)
# print(de_result.shape)


# 构建Generate类
class Generator(nn.Module):
    def __init__(self, d_model, vocab_size):
        # d_model : 代表词嵌入的维度
        # vovab_size : 代表词表的总大小
        super(Generator, self).__init__()
        # 定义一个线性层，完成网络输出维度的变换
        self.project = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # x ： 上一层的输出张量
        # 首先将x送入线性层中，让其经历softmax处理
        return F.log_softmax(self.project(x), dim=-1)


d_model = 512
vocab_size = 1000
x = de_result

gen = Generator(d_model, vocab_size)
gen_result = gen(x)
print(gen_result)
print(gen_result.shape)



tensor([[[-6.8956, -6.5986, -8.4811,  ..., -7.2004, -6.7450, -6.9793],
         [-7.0791, -5.8751, -8.2197,  ..., -6.8843, -7.3700, -6.7006],
         [-6.8995, -6.8604, -7.5096,  ..., -6.8115, -6.6190, -7.3020],
         [-7.4304, -7.7939, -7.3737,  ..., -7.5600, -7.7729, -7.5911]],

        [[-7.0649, -6.2719, -6.9616,  ..., -5.7521, -6.9926, -7.0587],
         [-7.0456, -7.1196, -6.3995,  ..., -6.9879, -6.6631, -6.9461],
         [-6.1350, -7.3294, -6.2949,  ..., -6.6793, -6.9646, -7.6704],
         [-6.4270, -6.7152, -7.8952,  ..., -7.8854, -6.6828, -7.8045]]],
       grad_fn=<LogSoftmaxBackward>)
torch.Size([2, 4, 1000])


In [ ]:
# 构建编码器- 解码器结构类
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, source_embed, target_embed, generator):
        # encoder : 编码器对象
        # decoder : 解码器对象
        # source_embed : 源数据的嵌入函数
        # target_embed : 目标数据的嵌入函数
        # generator : 输出部分类别生成器 对象
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_emned = source_embed
        self.tgt_embed = target_embed
        self.generator = generator

    def forward(self, source, target, source_mask, target_mask):
        # source : 代表源数据
        # target : 代表目标数据
        # source_mask : 代表源数据的掩码张量
        # target_mask : 代表目标数据的掩码张量
        return self.decode(self.encode(source, source_mask), source_mask,
                           target, target_mask)

    def encode(self, source, source_mask):
        return self.encoder(self.src_emned(source), source_mask)

    def decode(self, memory, source_mask, target, target_mask):
        # memory : 代表经历编码器编码后的输出张量
        return self.decoder(self.tgt_embed(target), memory, source_mask, target_mask)


In [ ]:
# 构建编码器- 解码器结构类
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, source_embed, target_embed, generator):
        # encoder : 编码器对象
        # decoder : 解码器对象
        # source_embed : 源数据的嵌入函数
        # target_embed : 目标数据的嵌入函数
        # generator : 输出部分类别生成器 对象
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_emned = source_embed
        self.tgt_embed = target_embed
        self.generator = generator

    def forward(self, source, target, source_mask, target_mask):
        # source : 代表源数据
        # target : 代表目标数据
        # source_mask : 代表源数据的掩码张量
        # target_mask : 代表目标数据的掩码张量
        return self.decode(self.encode(source, source_mask), source_mask,
                           target, target_mask)

    def encode(self, source, source_mask):
        return self.encoder(self.src_emned(source), source_mask)

    def decode(self, memory, source_mask, target, target_mask):
        # memory : 代表经历编码器编码后的输出张量
        return self.decoder(self.tgt_embed(target), memory, source_mask, target_mask)


vocab_size = 1000
d_model = 512
encoder = en
decoder = de
source_embed = nn.Embedding(vocab_size, d_model)
target_embed = nn.Embedding(vocab_size, d_model)
generator = gen

source = target = Variable(torch.LongTensor([[100, 2, 421, 508], [491, 998, 1, 221]]))
source_mask = target_mask = Variable(torch.zeros(2, 4, 4))

ed = EncoderDecoder(encoder, decoder, source_embed, target_embed, generator)
ed_result = ed(source, target, source_mask, target_mask)
print(ed_result)
print(ed_result.shape)



tensor([[[ 1.2461, -1.9424, -0.2390,  ...,  0.3894, -2.0771,  0.4588],
         [ 2.1275, -1.4796, -0.1383,  ...,  0.5331, -1.5249,  0.6149],
         [ 0.2116, -0.6087, -0.3678,  ..., -0.2838, -2.2089,  0.3160],
         [ 1.4840, -1.8253, -0.7596,  ..., -0.3792, -1.7015, -0.0275]],

        [[ 0.4927,  0.2528,  0.9127,  ..., -0.1492, -1.7515, -0.5555],
         [ 0.6065, -0.7407, -0.4137,  ..., -0.4184, -0.7686, -0.3387],
         [ 0.1525, -0.6770, -0.3392,  ...,  0.0147, -1.3579, -1.0997],
         [ 0.3306,  0.6021, -0.1215,  ..., -0.0857, -1.4704, -1.1116]]],
       grad_fn=<AddBackward0>)
torch.Size([2, 4, 512])


In [ ]:
def make_model(source_vocab, target_vocab, N=6, d_model=512, d_ff=2048, head=8, dropout=0.1):
    # source_vocab : 源数据的词汇总数
    # target_vocab : 目标数据的词汇总数
    # N : 编码器和解码器堆叠的层数
    # d_model : 词嵌入的维度
    # d_ff : 前馈全连接层中变换矩阵的维度
    # head : 多头注意力机制头数
    # dropout :  置零比率
    c = copy.deepcopy

    # 实例化一个多头注意力类
    attn = MultiHeadAttention(head, d_model)

    # 实例化一个前馈全连接层的网络对象
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)

    # 实例化一个位置编码器
    position = PositionalEncoding(d_model, dropout)

    # 实例化模型model, 利用的是Encoder和Decoder类
    # 编码器的结构里面有2个子层，attention层 和 前馈全连接层
    # 解码器的结构中有3 个子层，两个attention 层和前馈全连接层

    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embedding(d_model, source_vocab), c(position)),
        nn.Sequential(Embedding(d_model, target_vocab), c(position)),\
        Generator(d_model, target_vocab)
    )

    # 初始化整个模型中的参数，如果参数的维度大于1，将矩阵初始化成一个服从均匀分布的矩阵
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)

    return model


In [ ]:
# 构建数据生成器
def data_generator(V, batch_size, num_batch):
    # 随机生成数据的最大值=1, 假设V=101， 生成的数据是 1-100
    # batch_size : 每次喂给模型的样本数量
    # num_batch ；一共喂模型 多少轮
    for i in range(num_batch):
        # 使用numpy中的random.randint()来随机生产 [1, V)
        # 分布的形状是 (batch, 10)
        data = torch.from_numpy(np.random.randint(1, V, size=(batch_size, 10)))

        # 将数据的一列全部设置为1， 作为起始标志
        data[:, 0] = 1

        # 因为是copy任务，所以源数据和目标数据完全一致
        # 设置参数 requires_grad=False，样本的参数不需要参与梯度的计算
        source = Variable(data, requires_grad=False)
        target = Variable(data, requires_grad=False)

        yield Batch(source, target)


In [ ]:
from pyitcast.transformer_utils import Batch
from pyitcast.transformer_utils import get_std_opt
from pyitcast.transformer_utils import LabelSmoothing
from pyitcast.transformer_utils import SimpleLossCompute
from pyitcast.transformer_utils import run_epoch
from pyitcast.transformer_utils import greedy_decode

V = 11
batch_size = 20
num_batch = 30

# 首先使用make_model()函数 生成模型的实例化对象
model = make_model(V, V, N=2)

# 使用工具包 get_std-opt 获得模型的优化器
model_optimizer = get_std_opt(model)

# 使用labelSmoothing 获得标签平滑对象
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)

# 使用工具包 SimpleLossCompute 获得利用标签平滑的结果，得到损失计算方法
loss = SimpleLossCompute(model.generator, criterion, model_optimizer)
